<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing Smolagents with Arize Phoenix</h1>

## Install Dependencies

In [ ]:
!pip install smolagents -q
!pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

## Connect to Phoenix


In this example, we'll use Phoenix as our destination. You could instead add any other exporters you'd like in this approach.

If you need to set up an API key for Phoenix, you can do so [here](https://app.phoenix.arize.com/).

#### Keys

Note: For this colab you'll need:

*   Phoenix API key
*   Hugging Face Token

In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass.getpass("Enter your Phoenix API Key: ")
if not (hf_token_value := os.getenv("HF_TOKEN")):
    hf_token_value = getpass.getpass("Enter your Hugging Face Token: ")

# Set the environment variables with the provided keys
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={phoenix_api_key}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
os.environ["HF_TOKEN"] = f"{hf_token_value}"

Enter your Phoenix API Key: ··········
Enter your Hugging Face Token: ··········


### Define the Trace Provider

In [ ]:
from phoenix.otel import register

tracer_provider = register(
  project_name="my-agent-app", # Default is 'default'
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-agent-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Instrument Smolagents




In [ ]:
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

## Create your Smolagents & Run


Now you can create any sort of agent using the smolagents module, and Phoenix will track every step, input, output, tool, and call your agent makes.

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    ManagedAgent,
    DuckDuckGoSearchTool,
    VisitWebpageTool,
    HfApiModel,
)

model = HfApiModel()

agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
)
managed_agent = ManagedAgent(
    agent=agent,
    name="managed_agent",
    description="This is an agent that can do web search.",
)
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[managed_agent],
)

manager_agent.run(
    "What are Smolagents?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are Smolagents?                                                                                            │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  smolagents_info = managed_agent(request="What are Smolagents? Provide a detailed definition and context.")       
  print(smolagents_info)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'managed_agent'.                                                                   │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ What are Smolagents? Provide a detailed definition and context.                                                 │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│ {additional_prompting}                                                                                          │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Smolagents is a minimalist AI agent framework developed by Hugging Face, enabling developers to    
  create powerful agents with minimal code. It focuses on simplicity and efficiency, making it easy to deploy      
  agents that can interact with the real world using large language models (LLMs).")                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Smolagents is a minimalist AI agent framework developed by Hugging Face, enabling developers to
create powerful agents with minimal code. It focuses on simplicity and efficiency, making it easy to deploy agents 
that can interact with the real world using large language models (LLMs).

[Step 1: Duration 2.03 seconds| Input tokens: 5,155 | Output tokens: 461]

'Smolagents is a minimalist AI agent framework developed by Hugging Face, enabling developers to create powerful agents with minimal code. It focuses on simplicity and efficiency, making it easy to deploy agents that can interact with the real world using large language models (LLMs).'

 ## ⭐⭐⭐ If you like this guide, please give [Arize Phoenix](https://github.com/Arize-ai/phoenix) a star ⭐⭐⭐